In [ ]:

import optuna
import pandas as pd

In [ ]:
study_names = optuna.study.get_all_study_names(storage="sqlite:///../exhaustive.db")

In [ ]:
study = optuna.load_study(study_name="vgg16_grid_1_pruning_False", storage="sqlite:///../exhaustive.db")

In [ ]:
df = study.trials_dataframe()
df = df[df['state'] == 'COMPLETE']

In [ ]:
df

# Exploratory Figure 1

In [ ]:
df_fig1 = df[(df['params_layer'] == 22) & (df['params_edge-accelerator'] == 'off')]
df_fig1 = df_fig1[['values_energy in mJ', 'values_latency in ms', 'params_cpu-freq']]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Define font size variables
scale_fonts = 4
label_font_size = 14 + scale_fonts
title_font_size = 16 + scale_fonts
tick_font_size = 12 + scale_fonts
legend_font_size = 12 + scale_fonts
offset_font_size = 12 + scale_fonts

# Group by 'params_cpu-freq' and calculate the mean of 'values_energy in mJ' and 'values_latency in ms'
grouped_df = df_fig1.groupby('params_cpu-freq').max().reset_index()

# Create the plot
fig, ax1 = plt.subplots(figsize=(5, 4))
ax2 = ax1.twinx()

# Plot the latency on the first y-axis
ax1.set_xlabel('CPU Frequency (MHz)', fontsize=label_font_size)
ax1.set_ylabel('Latency (ms)', fontsize=label_font_size)
latency_line, = ax1.plot(grouped_df['params_cpu-freq'], grouped_df['values_latency in ms'], color='tab:red', marker='o',
                         linestyle='-')
ax1.tick_params(axis='y', labelsize=tick_font_size)
ax1.tick_params(axis='x', labelsize=tick_font_size)

# Set x-axis ticks from 600 to 1800 with increments of 200
ax1.set_xticks(range(600, 1801, 200))

# Rotate x-axis labels
for label in ax1.get_xticklabels():
    label.set_rotation(30)

# Create a second y-axis for the energy
ax2.set_ylabel('Energy (mJ)', fontsize=label_font_size)
energy_line, = ax2.plot(grouped_df['params_cpu-freq'], grouped_df['values_energy in mJ'], color='tab:blue', marker='^',
                        linestyle='--')
ax2.tick_params(axis='y', labelsize=tick_font_size)


# Format y-axis ticks as scientific notation for both y-axes
ax1.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax1.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
ax2.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax2.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))

# Set offset text size for y-axes
ax1.yaxis.get_offset_text().set_fontsize(offset_font_size)
ax2.yaxis.get_offset_text().set_fontsize(offset_font_size)

# Add a single legend
lines = [latency_line, energy_line]
labels = ['Latency', 'Energy']
ax2.legend(lines, labels, loc='upper right', fontsize=legend_font_size)

# Adjust layout
fig.tight_layout()
#fig.subplots_adjust(bottom=0.2)  # Adjust the bottom to make room for rotated labels

# Save the plot
plt.savefig('cpu_settings_effect.pdf', format='pdf')

# Show the plot
plt.show()


# Exploratory Figure 2

In [ ]:
df_fig2 = df[(df['params_cpu-freq'] == 1800) & (df['params_edge-accelerator'] == 'max') & (
        df['params_server-accelerator'] == True)]
df_fig2 = df_fig2[['values_energy in mJ', 'values_latency in ms', 'params_layer']]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Define font size variables
scale_fonts = 4
label_font_size = 14 + scale_fonts
title_font_size = 16 + scale_fonts
tick_font_size = 12 + scale_fonts
legend_font_size = 12 + scale_fonts
offset_font_size = 12 + scale_fonts

# Group by 'params_cpu-freq' and calculate the mean of 'values_energy in mJ' and 'values_latency in ms'
grouped_df = df_fig2.groupby('params_layer').max().reset_index()

# Create the plot
fig, ax1 = plt.subplots(figsize=(5, 4))

# Plot the latency on the first y-axis
ax1.set_xlabel('Split Layer', fontsize=label_font_size)
ax1.set_ylabel('Latency (ms)', fontsize=label_font_size)
latency_line, = ax1.plot(grouped_df['params_layer'], grouped_df['values_latency in ms'], color='tab:red', marker='o',
                         linestyle='-')
ax1.tick_params(axis='y', labelsize=tick_font_size)
ax1.tick_params(axis='x', labelsize=tick_font_size)

# Remove scientific notation for x-axis ticks
ax1.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
ax1.ticklabel_format(style='plain', axis='x')

# Create a second y-axis for the energy
ax2 = ax1.twinx()
ax2.set_ylabel('Energy (mJ)', fontsize=label_font_size)
energy_line, = ax2.plot(grouped_df['params_layer'], grouped_df['values_energy in mJ'], color='tab:blue', marker='^',
                        linestyle='--')
ax2.tick_params(axis='y', labelsize=tick_font_size)

# Format y-axis ticks as scientific notation for both y-axes
ax1.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax1.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
ax2.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax2.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))

# Set offset text size for y-axes
ax1.yaxis.get_offset_text().set_fontsize(offset_font_size)
ax2.yaxis.get_offset_text().set_fontsize(offset_font_size)

# Add a single legend
lines = [latency_line, energy_line]
labels = ['Latency', 'Energy']
ax2.legend(lines, labels, loc='upper center', fontsize=legend_font_size)

# Adjust layout
fig.tight_layout()
#fig.subplots_adjust(bottom=0.2)  # Adjust the bottom to make room for rotated labels

# Save the plot
plt.savefig('split_layer_effect.pdf', format='pdf')

# Show the plot
plt.show()

# Exploratory Figure 3

In [ ]:
df_fig3 = df[(df['params_cpu-freq'] == 1800) & (df['params_layer'] == 22) & (df['params_server-accelerator'] == True)]
df_fig3 = df_fig3[['values_energy in mJ', 'values_latency in ms', 'params_edge-accelerator']]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Reorder the dataframe
df_fig3['params_edge-accelerator'] = pd.Categorical(df_fig3['params_edge-accelerator'],
                                                    categories=['off', 'std', 'max'], ordered=True)
df_fig3 = df_fig3.sort_values('params_edge-accelerator')

# Define font size variables
scale_fonts = 4
label_font_size = 14 + scale_fonts
title_font_size = 16 + scale_fonts
tick_font_size = 12 + scale_fonts
legend_font_size = 12 + scale_fonts
offset_font_size = 12 + scale_fonts

# Create the plot
fig, ax1 = plt.subplots(figsize=(5, 4))

# Set the width for the bars
bar_width = 0.35
positions = range(len(df_fig3))

# Plot the latency bars
latency_bars = ax1.bar([p - bar_width / 2 for p in positions], df_fig3['values_latency in ms'], width=bar_width,
                       color='tab:red')
ax1.set_ylabel('Latency (ms)', fontsize=label_font_size)
ax1.tick_params(axis='y', labelsize=tick_font_size)

# Create a secondary y-axis for the energy bars
ax2 = ax1.twinx()
energy_bars = ax2.bar([p + bar_width / 2 for p in positions], df_fig3['values_energy in mJ'], width=bar_width,
                      color='tab:blue', hatch="//")
ax2.set_ylabel('Energy (mJ)', fontsize=label_font_size)
ax2.tick_params(axis='y', labelsize=tick_font_size)

# Format y-axis ticks as scientific notation for both y-axes
ax1.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax1.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
ax2.yaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
ax2.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))

# Set offset text size for y-axes
ax1.yaxis.get_offset_text().set_fontsize(offset_font_size)
ax2.yaxis.get_offset_text().set_fontsize(offset_font_size)

# Set the x-axis
ax1.set_xticks(positions)
ax1.set_xticklabels(df_fig3['params_edge-accelerator'])
ax1.set_xlabel('TPU Settings', fontsize=label_font_size)
ax1.tick_params(axis='x', labelsize=tick_font_size)

# Add a single legend
lines = [latency_bars, energy_bars]
labels = ['Latency', 'Energy']
ax1.legend(lines, labels, loc='upper right', fontsize=legend_font_size)


# Show the plot
fig.tight_layout()
plt.savefig('tpu_settings_effect.pdf', format='pdf')

# Show the plot
plt.show()

# Exploratory Figure 4

In [ ]:
df_fig4 = df[
    ['values_accuracy', 'params_edge-accelerator', 'params_layer']]

In [ ]:
import matplotlib.pyplot as plt

# Define font size variables
scale_fonts = 4
label_font_size = 14 + scale_fonts
title_font_size = 16 + scale_fonts
tick_font_size = 12 + scale_fonts
legend_font_size = 12 + scale_fonts
offset_font_size = 12 + scale_fonts

# Prepare data for plotting
plot_data = df_fig4[df_fig4['params_edge-accelerator'].isin(['off', 'std'])].copy()

# Update the dataframe to rename 'std' to 'on' in the 'params_edge-accelerator' column
plot_data.loc[plot_data['params_edge-accelerator'] == 'std', 'params_edge-accelerator'] = 'on'

# Group by 'params_edge-accelerator' and 'params_layer' and calculate the mean of 'values_accuracy'
plot_data = plot_data.groupby(['params_edge-accelerator', 'params_layer']).mean().reset_index()

# Create a line plot to showcase the effect of layer and edge accelerator on accuracy
fig, ax = plt.subplots(figsize=(5, 4))

# Plot data for 'off' edge accelerator
off_data = plot_data[plot_data['params_edge-accelerator'] == 'off']
line1, = ax.plot(off_data['params_layer'], off_data['values_accuracy'], marker='^', color='tab:orange', linestyle='--')

# Plot data for 'on' edge accelerator
on_data = plot_data[plot_data['params_edge-accelerator'] == 'on']
line2, = ax.plot(on_data['params_layer'], on_data['values_accuracy'], marker='o', color='tab:green', linestyle='-')

ax.set_xlabel('Split Layer', fontsize=label_font_size)
ax.set_ylabel('Accuracy', fontsize=label_font_size)
ax.tick_params(axis='both', labelsize=tick_font_size)

# Create custom legend
custom_lines = [line2, line1]
ax.legend(custom_lines, ['TPU On', 'TPU Off'], fontsize=legend_font_size, loc='lower center')

fig.tight_layout()
plt.savefig('accuracy_effect.pdf', format='pdf', bbox_inches='tight')

# Show the plot
plt.show()
